In [1]:
from pathlib import Path

from IPython.display import Audio
import librosa
import numpy as np
import soundfile as sf

In [2]:
mediapath = Path().cwd().resolve().parent / "media"
list(mediapath.iterdir())


[PosixPath('/home/jim/projects/audio-processing/media/input5.wav'),
 PosixPath('/home/jim/projects/audio-processing/media/input6.wav'),
 PosixPath('/home/jim/projects/audio-processing/media/input1.wav'),
 PosixPath('/home/jim/projects/audio-processing/media/output1-rnn.wav')]

In [3]:
insample = mediapath / "input1.wav"
# outsample = mediapath / "output1-stft.wav"


In [4]:
Audio(insample)

In [5]:
y, sr = librosa.load(insample)
print(y.shape, sr)


(502740,) 22050


In [6]:
S_full, phase = librosa.magphase(librosa.stft(y))

S_filter = librosa.decompose.nn_filter(
    S_full,
    aggregate=np.median,
    metric="cosine",
    width=int(librosa.time_to_frames(2, sr=sr)),
)

S_filter = np.minimum(S_full, S_filter)
margin_i, margin_v = 1, 1
power = 1

mask_i = librosa.util.softmask(S_filter, margin_i * (S_full - S_filter), power=power)

mask_v = librosa.util.softmask(S_full - S_filter, margin_v * S_filter, power=power)


In [7]:
S_foreground = mask_v * S_full
S_background = mask_i * S_full
D_foreground = S_foreground * phase
y_foreground = librosa.istft(D_foreground)

Audio(y_foreground, rate=sr)
